In [2]:
import pandas as pd
import numpy as np
from avro.datafile import DataFileReader
from avro.io import DatumReader
import copy
import json
import base64

BACKUP_INTERMEDIATE_CONTAINER_NAME = "capture_processed"
MAX_EVENTS = 1000

In [3]:
# Hay que añadir el evento que envíe los resultados de la Partida
# Por ahora lo he añadido manualmente pero habría que enviarlo cuando el profesor finaliza el juego
questions_results_base64 = 'W3RydWUsZmFsc2UsZmFsc2UsdHJ1ZSx0cnVlXQ=='
questions_results_string = questions_results_base64.encode('ascii')
questions_results_string = base64.b64decode(questions_results_string)
questions_results_string = questions_results_string.decode('ascii')
questions_results = json.loads(questions_results_string)
questions_results

[True, False, False, True, True]

In [4]:
# Leemos los datos del fichero Avro
with open('upctevents/devevents/0/2023/07/13/14/28/05.avro', 'rb') as f:
    reader = DataFileReader(f, DatumReader())
    metadata = copy.deepcopy(reader.meta)
    schema_from_file = json.loads(metadata['avro.schema'])
    avro_data = [user for user in reader]
    reader.close()

# Creamos un dataframe con los datos que leemos del fichero avro sin filtrar datos
events = [json.loads(j['Body']) for j in avro_data]

df = pd.json_normalize(events)
df = df.drop(['domain'], axis=1)
df

,id,username,room,time,action_number,action,notes
0,a58a77da-0800-440b-9499-855ef04ee437,Alvaro,311567,3255,0,checkbox,"{""question"":""00"",""res"":true}"
1,a133a0f4-2267-40b8-9b3b-2bac0aa0cbbb,Prueba,311567,6397,0,checkbox,"{""question"":""00"",""res"":true}"
2,a133a0f4-2267-40b8-9b3b-2bac0aa0cbbb,Prueba,311567,31472,1,checkbox,"{""question"":""03"",""res"":true}"
3,a133a0f4-2267-40b8-9b3b-2bac0aa0cbbb,Prueba,311567,45440,2,checkbox,"{""question"":""04"",""res"":true}"
4,a133a0f4-2267-40b8-9b3b-2bac0aa0cbbb,Prueba,311567,51080,3,response,"{""res"":true,""score"":1}"
...,...,...,...,...,...,...,...
85,f2ebb0df-45aa-453c-a78d-f769ca1429ef,Alberto,411567,69983,2,checkbox,"{""question"":""01"",""res"":false}"
86,f2ebb0df-45aa-453c-a78d-f769ca1429ef,Alberto,411567,71701,3,checkbox,"{""question"":""01"",""res"":true}"
87,f2ebb0df-45aa-453c-a78d-f769ca1429ef,Alberto,411567,118608,4,checkbox,"{""question"":""04"",""res"":true}"
88,f2ebb0df-45aa-453c-a78d-f769ca1429ef,Alberto,411567,122772,5,response,"{""res"":false,""score"":0.6}"


In [5]:
# Guardamos los datos de la columna 'notes' en un DataFrame
notes = [j['notes'] for j in events]
notes_jsons = []

# Usamos un Try/Except porque la función json.loads devuelve error en un json vacío (el campo notes de la acción ‘reset’ no contiene información)
for j in notes :
    try:
        notes_jsons.append(json.loads(j))
    except (Exception):
        notes_jsons.append(np.NaN)

df_notes = pd.json_normalize(notes_jsons)
df_notes = df_notes.drop(['studentList'], axis=1)
df_notes

,question,res,score
0,00,True,NaN
1,00,True,NaN
2,03,True,NaN
3,04,True,NaN
4,NaN,True,1.0
...,...,...,...
85,01,False,NaN
86,01,True,NaN
87,04,True,NaN
88,NaN,False,0.6


In [6]:
# Incluimos estas columnas al DataFrame original
df = pd.concat([df, df_notes], axis=1)
df

,id,username,room,time,action_number,action,notes,question,res,score
0,a58a77da-0800-440b-9499-855ef04ee437,Alvaro,311567,3255,0,checkbox,"{""question"":""00"",""res"":true}",00,True,NaN
1,a133a0f4-2267-40b8-9b3b-2bac0aa0cbbb,Prueba,311567,6397,0,checkbox,"{""question"":""00"",""res"":true}",00,True,NaN
2,a133a0f4-2267-40b8-9b3b-2bac0aa0cbbb,Prueba,311567,31472,1,checkbox,"{""question"":""03"",""res"":true}",03,True,NaN
3,a133a0f4-2267-40b8-9b3b-2bac0aa0cbbb,Prueba,311567,45440,2,checkbox,"{""question"":""04"",""res"":true}",04,True,NaN
4,a133a0f4-2267-40b8-9b3b-2bac0aa0cbbb,Prueba,311567,51080,3,response,"{""res"":true,""score"":1}",NaN,True,1.0
...,...,...,...,...,...,...,...,...,...,...
85,f2ebb0df-45aa-453c-a78d-f769ca1429ef,Alberto,411567,69983,2,checkbox,"{""question"":""01"",""res"":false}",01,False,NaN
86,f2ebb0df-45aa-453c-a78d-f769ca1429ef,Alberto,411567,71701,3,checkbox,"{""question"":""01"",""res"":true}",01,True,NaN
87,f2ebb0df-45aa-453c-a78d-f769ca1429ef,Alberto,411567,118608,4,checkbox,"{""question"":""04"",""res"":true}",04,True,NaN
88,f2ebb0df-45aa-453c-a78d-f769ca1429ef,Alberto,411567,122772,5,response,"{""res"":false,""score"":0.6}",NaN,False,0.6


In [7]:
# Obtenemos la lista de juegos que se han jugado. Cada uno tiene una 'room' distinta
data = df.drop_duplicates(['username', 'room'])
rooms_data = data.loc[data['username'] == 'teacher']
rooms = rooms_data['room'].values
rooms

array(['311567', '411567'], dtype=object)

In [8]:
# Creamos una diccionario de dataframes usando como clave la 'room' de cada partida
# Estos dataframes contienen los resultados de esa partida
df_scores = {}
for room in rooms:
    # Buscamos el índice donde están los datos del profesor de este juego
    room_index = rooms_data.loc[rooms_data['room'] == room].index[0]

    # Filtramos por el campo 'notes' que es el que nos interesa
    room_notes = df.iloc[room_index]['notes']

    # Cargamos esta información en forma de JSON y creamos el DataFrame
    room_scores = json.loads(room_notes)['studentList']
    df_scores[room] = pd.json_normalize(room_scores)

# Finalmente mostramos el número de partidas que se están analizando
len(df_scores)

2

In [9]:
df_scores

{'311567':                                      id    res username   time  score  points
 0  a133a0f4-2267-40b8-9b3b-2bac0aa0cbbb   True   Prueba  51080    1.0    1000
 1  28f6fc69-f2d1-45d2-98cd-7779ac8edc83   True     Masi  71398    1.0     818
 2  9254e8de-105a-4d24-8cd1-9aef06852b80  False  Alberto  40839    0.8     800
 3  e433e084-7d17-4308-a031-c778b7a58b8e  False  Sarabia  47086    0.8     800
 4  356b7dc4-542e-43b3-aee7-84e5bb6fae07  False    David  47877    0.8     800
 5  37c79a25-ee62-4f15-be89-eb79f8e83630  False  Txentxo  73352    0.2     160,
 '411567':                                      id    res   username    time  score  \
 0  37a62cf4-c17e-4c59-a194-b82e8abdbe55   True       Pepe   56510    1.0   
 1  5e6c4f75-07cf-456b-a360-7d9f8ff1e7d1  False       Jose   74479    0.8   
 2  c173a975-5d89-4551-b432-b2d2550368f0  False    Serpent   84307    0.8   
 3  2b7ef295-d6e5-4b32-96f9-3f1e8d32d665  False  Alejandro  138481    0.8   
 4  db4bc498-fab9-42d9-8ff5-fce6504e338e 

In [10]:
# Resultados de la partida 311567
df_scores.get('311567')

,id,res,username,time,score,points
0,a133a0f4-2267-40b8-9b3b-2bac0aa0cbbb,True,Prueba,51080,1.0,1000
1,28f6fc69-f2d1-45d2-98cd-7779ac8edc83,True,Masi,71398,1.0,818
2,9254e8de-105a-4d24-8cd1-9aef06852b80,False,Alberto,40839,0.8,800
3,e433e084-7d17-4308-a031-c778b7a58b8e,False,Sarabia,47086,0.8,800
4,356b7dc4-542e-43b3-aee7-84e5bb6fae07,False,David,47877,0.8,800
5,37c79a25-ee62-4f15-be89-eb79f8e83630,False,Txentxo,73352,0.2,160


In [11]:
# Resultados de la partida 311567
df_scores.get('311567')
# Resultados de la partida 411567
df_scores.get(rooms[1])

,id,res,username,time,score,points
0,37a62cf4-c17e-4c59-a194-b82e8abdbe55,True,Pepe,56510,1.0,1000
1,5e6c4f75-07cf-456b-a360-7d9f8ff1e7d1,False,Jose,74479,0.8,773
2,c173a975-5d89-4551-b432-b2d2550368f0,False,Serpent,84307,0.8,759
3,2b7ef295-d6e5-4b32-96f9-3f1e8d32d665,False,Alejandro,138481,0.8,679
4,db4bc498-fab9-42d9-8ff5-fce6504e338e,False,Jose Luis,95663,0.6,557
5,8fb5035a-6bc7-4d1b-9ab1-912a9a7e4968,False,Guille,109367,0.6,541
6,f2ebb0df-45aa-453c-a78d-f769ca1429ef,False,Alberto,122772,0.6,526
7,23730f42-9ba7-4d2f-8454-002f1f43f6d5,False,Juanito,2436,0.4,400
8,4fa15f3a-4619-4ce1-b85d-d8168a519438,False,Victor,164651,0.4,320


In [12]:
# Eliminamos a la gente que no ha enviado su respuesta del DataFrame
df_std = pd.DataFrame()
for room in rooms:
    # Obtenemos la lista de usuarios que han respondido
    students_scores = df_scores.get(room)['username'].values
    
    # Filtramos el DataFrame por la sala y comprobamos si esta el usuario en la lista
    df_filter = df.loc[df['room'] == room][df['username'].isin(students_scores).dropna()]
    df_std = pd.concat([df_std, df_filter], axis=0)

# Eliminamos el campo 'notes' para no tenerlo duplicado
df_std = df_std.drop('notes', axis=1)
df_std = df_std.reset_index(drop=True)
df_std

C:\Users\javis\AppData\Local\Temp\ipykernel_10672\3645492748.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filter = df.loc[df['room'] == room][df['username'].isin(students_scores).dropna()]
C:\Users\javis\AppData\Local\Temp\ipykernel_10672\3645492748.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filter = df.loc[df['room'] == room][df['username'].isin(students_scores).dropna()]


,id,username,room,time,action_number,action,question,res,score
0,a133a0f4-2267-40b8-9b3b-2bac0aa0cbbb,Prueba,311567,6397,0,checkbox,00,True,NaN
1,a133a0f4-2267-40b8-9b3b-2bac0aa0cbbb,Prueba,311567,31472,1,checkbox,03,True,NaN
2,a133a0f4-2267-40b8-9b3b-2bac0aa0cbbb,Prueba,311567,45440,2,checkbox,04,True,NaN
3,a133a0f4-2267-40b8-9b3b-2bac0aa0cbbb,Prueba,311567,51080,3,response,NaN,True,1.0
4,e433e084-7d17-4308-a031-c778b7a58b8e,Sarabia,311567,10445,0,checkbox,02,True,NaN
...,...,...,...,...,...,...,...,...,...
82,f2ebb0df-45aa-453c-a78d-f769ca1429ef,Alberto,411567,56599,1,checkbox,01,True,NaN
83,f2ebb0df-45aa-453c-a78d-f769ca1429ef,Alberto,411567,69983,2,checkbox,01,False,NaN
84,f2ebb0df-45aa-453c-a78d-f769ca1429ef,Alberto,411567,71701,3,checkbox,01,True,NaN
85,f2ebb0df-45aa-453c-a78d-f769ca1429ef,Alberto,411567,118608,4,checkbox,04,True,NaN


In [13]:
# Funcion que devuelve todos los datos de un alumno en una partida 'room'
def student_data (id: str, room: str) -> dict:
    student_info = {}
    questions = 5

    # Recopilamos la informacion del estudiante
    df_student = df_std.loc[(df_std['id'] == id) & (df_std['room'] == room)]
    
    # Nota del estudiante
    score = df_student.loc[df_student['action'] == 'response']['score'].values[0]

    # Orden de las respuestas (-1 para el reset)
    questions_order = df_student.loc[(df_student['action'] == 'checkbox') | (df_student['action'] == 'reset')]['question'].fillna(-1).values.astype(int)

    # Comprobamos si el estudiante ha interactuado con las preguntas de la web
    if len(questions_order) == 0:
        # Si no ha respondido simplemente enviamos la nota
        student_info['Nota'] = score
        student_info['Número de Aciertos'] = int(score * questions)

    # Si ha respondido calculamos el resto de datos
    else:
        # Tiempo entre las respuestas
        times = df_student.loc[(df_student['action'] == 'checkbox') | (df_student['action'] == 'reset')]['time'].values.astype(int)
        time = []
        time.append(int(times[0]))
        for i in range(1, len(times)):
            time.append(int(times[i] - times[i - 1]))

        # Preguntas que ha contestado mas de una vez
        unique_responses, count = np.unique(questions_order, return_counts=True)
        duplicated_response = unique_responses[count > 1]
        # Comprobamos si ha pulsado el boton de reset
        reset = np.any(questions_order == -1)
        if np.any(questions_order == -1):
            duplicated_response = np.append(duplicated_response, -1)
        
        # Número de veces en las que ha cambiado su respuesta sin contar el reset
        number_resets = 0
        if reset:
            # Numero de veces que ha reseteado
            number_resets = count[np.where(unique_responses == -1)[0]]

            # Si ha reseteado varias veces restamos el número de reseteos
            if number_resets > 1:
                n_duplicated_response = np.sum(count[count > 1] - 1) - (number_resets - 1)
            # En caso contrario simplemente sumamos las veces que ha pulsado las preguntas mas de 1 vez y restamos esa primera pulsacion
            else:
                n_duplicated_response = np.sum(count[count > 1] - 1)
        else:
            n_duplicated_response = np.sum(count[count > 1] - 1)


        # Resultado de cada cuestión
        df_responses = df_student.loc[(df_student['action'] == 'checkbox') | (df_student['action'] == 'reset')]['question'].fillna(-1).values.astype(int)
        responses = {}
        for i in df_responses:
            i = int(i)
            # Si ha reseteado limpiamos la lista
            if (i == -1):
                responses.clear()
            elif (i in responses):
                if (responses[i] == True):
                    responses[i] = False
                else:
                    responses[i] = True
            else:
                responses[i] = True

        # Rellenamos el resto de respuestas a False
        for i in range(questions):
            if(i not in responses):
                responses[i] = False

        # Ordenamos las cuestiones
        sorted_responses = dict(sorted(responses.items()))


        # Comprobamos si ha acertado su respuesta
        success = list(range(questions))
        for i in range(questions):
            if (sorted_responses[i] == questions_results[i]):
                success[i] = True
            else:
                success[i] = False
        
            


        # Introducimos la información en el diccionario
        student_info['Nota'] = score
        student_info['Número de Aciertos'] = int(score * questions)

        ################## No podemos saber las respuestas que ha dejado en Falso ni cuanto ha tardado en responder esa pregunta ##################
        student_info['Orden de respuestas'] = questions_order.tolist()
        student_info['Tiempo entre las respuestas'] = time
        student_info['Preguntas que ha contestado mas de una vez'] = duplicated_response.tolist()
        student_info['Número de preguntas en las que ha cambiado su respuesta'] = int(n_duplicated_response)
        student_info['El estudiante ha reseteado'] = bool(reset)
        student_info['Ha acertado la cuestión'] = success
        student_info['Respuestas del estudiante'] = sorted_responses

    return student_info

In [14]:
# Información del estudiante Sarabia de la partida 311567
student_data('e433e084-7d17-4308-a031-c778b7a58b8e', '311567')

{'Nota': 0.8,
 'Número de Aciertos': 4,
 'Orden de respuestas': [2, 2, 3, 0, -1, 3, 0],
 'Tiempo entre las respuestas': [10445, 515, 12596, 2044, 3781, 7601, 3048],
 'Preguntas que ha contestado mas de una vez': [0, 2, 3, -1],
 'Número de preguntas en las que ha cambiado su respuesta': 3,
 'El estudiante ha reseteado': True,
 'Ha acertado la cuestión': [True, True, True, True, False],
 'Respuestas del estudiante': {0: True, 1: False, 2: False, 3: True, 4: False}}

In [15]:
# Utilizamos de ejemplo el nombre de los estudiantes, pero la función utiliza el id
student_data('28f6fc69-f2d1-45d2-98cd-7779ac8edc83', '311567')

{'Nota': 1.0,
 'Número de Aciertos': 5,
 'Orden de respuestas': [0, 3, 4],
 'Tiempo entre las respuestas': [10960, 34763, 22034],
 'Preguntas que ha contestado mas de una vez': [],
 'Número de preguntas en las que ha cambiado su respuesta': 0,
 'El estudiante ha reseteado': False,
 'Ha acertado la cuestión': [True, True, True, True, True],
 'Respuestas del estudiante': {0: True, 1: False, 2: False, 3: True, 4: True}}

In [16]:
# Crearme una funcion que sabiendo la sala y pregunta me devuelva los resultados de cada estudiante para esa pregunta.
def question_responses (room: str, df_question: pd.DataFrame) -> pd.DataFrame:
    question_results = pd.DataFrame(columns=['id', 'username', 'response', 'changes', 'last_action'])

    # Obtenemos la lista de estudiantes que han participado
    students = df_scores.get(room)['username'].values
    room_data = df_scores.get(room)

    for s in students:
        # Comprobamos si el estudiante ha cambiado la pregunta
        check = (df_question['username'] == s).any()
        if check:
            # Nos guardamos los eventos de este usuario
            user_events = df_question.loc[df_question['username'] == s]
            result = False
            changes = 0

            # Comprobamos lo que ha hecho en ese evento y cambiamos el resultado de la pregunta
            for index, row in user_events.iterrows():
                action = row['action']
                action_number = row['action_number'] + 1
                changes += 1
                if action == 'checkbox':
                    if result:
                        result = False
                    else:
                        result = True
                elif action == 'reset':
                    result = False
            
            # Creamos un DataFrame temporal con el usuario y la respuesta y lo añadimos al DataFrame de respuestas
            df_temp = pd.DataFrame({'id': room_data.loc[room_data['username'] == s]['id'].values,
                                    'username': s,
                                    'response': result,
                                    'changes': changes,
                                    'last_action': action_number})
            question_results = pd.concat([question_results, df_temp], ignore_index=True)
        else:
            # Si no ha cambiado esa pregunta el valor por defecto es False
            # Creamos un DataFrame temporal con el usuario y la respuesta y lo añadimos al DataFrame de respuestas
            df_temp = pd.DataFrame({'id': room_data.loc[room_data['username'] == s]['id'].values,
                                    'username': s,
                                    'response': False,
                                    'changes': 0,
                                    'last_action': -1})
            question_results = pd.concat([question_results, df_temp], ignore_index=True)

    return question_results

In [17]:
# Función que devuelve todos los datos de una pregunta en una partida 'room'
def question_data (question: int, room: str) -> dict:
    question_info = {}
    
    # Recopilamos la información de la pregunta
    df_question = df_std.loc[((df_std['question'] == '0' + str(question)) | (df_std['action'] == 'reset'))& (df_std['room'] == room)]

    # Número de estudiantes que han participado en la partida
    number_students = df_std.loc[df_std['room'] == room]['username'].nunique()

    # Comprobamos si se ha hecho alguna modificación en esta pregunta
    if (df_question['action'] == 'checkbox').any() == False:
        # No hay cambios en la pregunta
        question_info['Número de usuarios que han votado'] = 0
        question_info['Número de usuarios que han votado True'] = 0
        question_info['Número de usuarios que han votado False'] = 0
    else:
        # Hay cambios en la pregunta

        # Obtenemos los resultados de la pregunta con la respuesta de todos los estudiantes
        df_results = question_responses(room, df_question)

        # Obtenemos los resultados de los usuarios
        df_no_reset = df_results[df_results['last_action'] != -1]
        std_results = df_no_reset['response'].value_counts()

        # Contamos el número de respuestas True y False
        # Usamos excepciones por si no hay resultados de ese tipo
        try:
            std_true = std_results[True]
        except:
            std_true = 0

        # Como hay alumnos que dejan la respuesta en falso, restamos el numero de alumnos totales al numero de alumnos que han contestado True
        std_false = number_students - std_true
        try:
            std_responded_false = std_results[False]
        except:
            std_responded_false = 0

        # Obtenemos los tiempos de los usuarios en responder
        times = df_question.groupby('username')['time'].last().values.astype(int)
        mean = np.mean(times)

        # Obtenemos la acción en la que los usuarios han respondido esta pregunta
        unique_responses, count = np.unique(df_no_reset['last_action'], return_counts=True)

        responses_count = {}
        for i in range(0, len(unique_responses)):
            responses_count[unique_responses[i]] = int(count[i])

        # Guardamos toda la información en el diccionario
        question_info['Número de usuarios que han cambiado el valor de esta cuestión'] = int(std_true + std_responded_false)
        question_info['Número de usuarios que han respondido True'] = int(std_true)
        question_info['Número de usuarios que han respondido False'] = int(std_false)
        question_info['Número de usuarios que han dejado la respuesta en False'] = int(std_false - std_responded_false)

        # Comprobamos si la respuesta de esa pregunta es verdadera o falsa
        if questions_results[question]:
            question_info['Porcentaje de aciertos'] = float("{:.2f}".format((std_true/(std_false + std_true)) * 100))
        else:
            question_info['Porcentaje de aciertos'] = float("{:.2f}".format((std_false/(std_false + std_true)) * 100))
        question_info['Tiempos de respuesta'] = times.tolist()
        question_info['Tiempo medio de respuesta'] = mean
        question_info['Acción en la que responden los usuarios'] = responses_count
    
    return question_info

In [23]:
question_data(3, '311567')

{'Número de usuarios que han cambiado el valor de esta cuestión': 5,
 'Número de usuarios que han respondido True': 4,
 'Número de usuarios que han respondido False': 2,
 'Número de usuarios que han dejado la respuesta en False': 1,
 'Porcentaje de aciertos': 66.67,
 'Tiempos de respuesta': [34927, 45723, 31472, 36982, 65508],
 'Tiempo medio de respuesta': 42922.4,
 'Acción en la que responden los usuarios': {2: 2, 3: 1, 5: 1, 6: 1}}

In [19]:
question_data(1, '311567')

{'Número de usuarios que han cambiado el valor de esta cuestión': 3,
 'Número de usuarios que han respondido True': 2,
 'Número de usuarios que han respondido False': 4,
 'Número de usuarios que han dejado la respuesta en False': 3,
 'Porcentaje de aciertos': 66.67,
 'Tiempos de respuesta': [40093, 29381, 35084],
 'Tiempo medio de respuesta': 34852.666666666664,
 'Acción en la que responden los usuarios': {1: 1, 5: 1, 6: 1}}

In [20]:
# Finalmente guardamos toda la inforamción en 2 ficheros JSON.

# Obtenemos la lista de todos los estudiantes y la partida
students = df_std.drop_duplicates(['id', 'room'])[['id', 'username', 'room']]
student_info = {}

for i in range(len(students)):
    # Obtenemos el usuario y su partida para pasarselos a la funcion
    user = students.iloc[i]['username']
    id = students.iloc[i]['id']
    r = students.iloc[i]['room']

    student_info[f'{user}/**/{id}/**/{r}'] = student_data(id, r)

# Guardamos la info en un archivo local
with open('students_data.json', 'w') as outfile:
    json.dump(student_info, outfile)

In [21]:
# Hacemos lo mismo para guardar la informacion de las preguntas
question_info = {}

# Recorremos todas las preguntas de todas las partidas
for r in rooms:
    # Como no sabemos el numero de preguntas que hay en la sala, tenemos que realizar un try except para manejar excepciones
    try:
        check = 0
        for i in range(0, 999):
            q = question_data(i, r)

            # Comprobamos si existe el campo 'Porcentaje de aciertos' para ver si hay respuesta en esta pregunta
            prueba = q['Porcentaje de aciertos']
            question_info[f'{i}/**/{r}'] = question_data(i, r)
            check = 0
    
    except:
        # Esta excepción puede saltar si nadie ha respondido esa pregunta, si salta varias veces seguidas paramos el bucle
        check += 1
        if check <= 3:
            continue


# Guardamos la info en un archivo local
with open('questions_data.json', 'w') as outfile:
    json.dump(question_info, outfile)